# SuggestBot Data Extraction

## Setup

In [1]:
import datetime
from dateutil import parser
import re

import mwparserfromhell
import wmfdata

In [2]:
spark = wmfdata.spark.get_session(app_name='pyspark reg; wikidata props',
                                  type='yarn-regular', # local, yarn-regular, yarn-large
                                  )  

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [3]:
print("Mediawiki partitions:")
spark.sql("show partitions wmf.mediawiki_wikitext_history PARTITION(wiki_db='enwiki')")

print("\nWikidata partitions:")
spark.sql("SHOW PARTITIONS wmf.wikidata_item_page_link").show(50, False)

gen_table = 'isaacj.gender_wikidata'
geo_table = 'isaacj.qid_to_country'
print("\nGeo partitions:")
spark.sql(f"SHOW PARTITIONS {geo_table}").show(50, False)

print("\nGender partitions:")
spark.sql(f"SHOW PARTITIONS {gen_table}").show(50, False)


Mediawiki partitions:
['snapshot=2021-10', 'snapshot=2021-11', 'snapshot=2021-12']

Wikidata partitions:
+-------------------+
|partition          |
+-------------------+
|snapshot=2021-11-29|
|snapshot=2021-12-06|
|snapshot=2021-12-13|
|snapshot=2021-12-20|
|snapshot=2021-12-27|
|snapshot=2022-01-03|
+-------------------+


Geo partitions:
+-------------------+
|partition          |
+-------------------+
|snapshot=2022-01-03|
+-------------------+


Gender partitions:
+-------------------+
|partition          |
+-------------------+
|snapshot=2022-01-03|
+-------------------+



In [7]:
mw_snapshot = '2021-12'
wd_snapshot = '2022-01-03'
geo_snapshot = '2022-01-03'
gen_snapshot = '2022-01-03'
start_date = '2021-01-01'
wiki_db = 'enwiki'
tablename = 'isaacj.suggestbot_recs_2021'

## Functions

In [8]:
def extract_user_title(talkpage_title):
    try:
        return talkpage_title.split(':', 1)[1].split('/')[0]  # remove User_talk: and any subpage info
    except Exception:
        return talkpage_title  # if error, just return whole page title
    
spark.udf.register('extract_user_title', extract_user_title, 'String')

<function __main__.extract_user_title(talkpage_title)>

In [9]:
def extract_recs(wikitext):
    """Extract list of SuggestBot recommendations + metadata from talk page wikitext.

    Get the most recent SuggestBot section (sometimes there are more than 1 SuggestBot sections on a page).

    Example output in format '<rank>|<predicted-quality>|<article-title>' based on above wikitext:
    ['1|Medium|Pathological lying|Add sources', '2|Medium|Employee assistance program|Add sources', ...]

    The pipe-delimited string makes it easier to handle in Spark.
    I can then run this on a given month and output a TSV of the format:
    User Timestamp Rank Quality TaskType PageTitle PageID Gender Geography

    And you can gather each editor's history from that time-period and compare to the recommendations compute uptake.
    """
    try:
        col_title = []
        col_views = []
        col_quality = []
        col_assessed_class = []
        col_predicted_class = []
        col_tag = []
        wt = mwparserfromhell.parse(wikitext)
        sections = wt.get_sections()
        timestamps = []
        found = False # flag that changes when we find a SuggestBot post
        # Iterate through sections until we find the most recent SuggestBot table
        for s in sections:
            time = datetime.datetime(1, 1, 1, 0, 0)  # dummy timestamp of 0 if not SuggestBot
            # If it's a SuggestBot section, update timestamp and set "found" to true.
            if(str(s).startswith('== Articles you might like to edit, from SuggestBot ==')):
                try:
                    # Update regex that enforces SuggestBot's account (in case timestamp somehow in e.g., article title)
                    # and ends with the (UTC) declaration because spacing/text after the timestamp is unpredictable as
                    # other users could comment on SuggestBot's recommendations so no guarantee they are the whole section
                    ts = re.search('(?<=\(\[\[User talk\:SuggestBot\|talk\]\]\) )[0-9][0-9]\:[0-9][0-9]\,.*(?= \(UTC\))', str(s)).group(0)
                    time = parser.parse(ts)
                    found = True
                except (AttributeError, ParserError):
                    pass
            timestamps.append(time)
        # If we've found at least one SuggestBot table, run code below. Otherwise return None.
        if found:
            s = sections[timestamps.index(max(timestamps))] # Find most recent
            table = None
            for tag in s.filter_tags():
                if tag.tag == 'table':
                    table = str(tag)
                    break
            if table is None:
                return None
            
            col_title = [re.sub('\| \[\[', '', re.sub('\]\] \<small\>','',x)) for x in re.findall('\| \[\[.*\]\] \<small\>', table)]
            col_views = [re.sub('\{\{formatnum\:', '', re.sub('\}\}','',x)) for x in re.findall('\{\{formatnum:[0-9]*\}\}', table)]
            col_quality = [re.sub('alt\=Quality\: ', '', re.sub(',','',x)) for x in re.findall('alt\=Quality\: [A-z]*,', table)]
            col_assessed_class = [re.sub('alt\=Quality\: [A-z]* Assessed class\: ', '', re.sub(',','',x)) for x in re.findall('alt\=Quality\: [A-z]*, Assessed class\: [A-z]*,', table)]
            col_predicted_class = [re.sub('Predicted class\: ', '', re.sub('\]','',x)) for x in re.findall('Predicted class\: [A-z]*\]', table)]
            col_tag = [re.sub('\| ', '', re.sub('\n\n','',x)) for x in re.findall('\| [A-z| ]*\n\n', table)]
            col_tag.append(re.sub('\| ', '', re.sub('\n.*','',re.findall('\| [A-z| ]*\n\|\}', table)[0])))
            col_rank = [r for r in range(1, len(col_title) + 1)]

            pipe_delimited = [[int(rank), str(title), int(views), str(quality), str(assessed_class), str(predicted_class), str(tag)] for rank,title,views,quality,assessed_class,predicted_class,tag in zip(col_rank, col_title, col_views, col_quality, col_assessed_class, col_predicted_class, col_tag)]
            return pipe_delimited
        else:
            return None
    except Exception:
        return None
    
spark.udf.register('extract_recs', extract_recs, 'Array<STRUCT<rank:Int, title:String, views:Int, quality:String, assess_class:String, pred_class:String, tag:String>>')

<function __main__.extract_recs(wikitext)>

## Gather data

In [10]:
create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {tablename} (
        wiki_db                         STRING        COMMENT 'Wiki -- e.g., enwiki for English Wikipedia',
        user_text                       STRING        COMMENT 'User text receiving recommendations',
        revision_timestamp              STRING        COMMENT 'Revision timestamp (UTC) when recommendations were made',
        revision_id                     BIGINT        COMMENT 'Revision ID when recommendations were made',
        rank                            INT           COMMENT 'Rank of recommendation within table of 30',
        title                           STRING        COMMENT 'Recommended page title',
        views                           INT           COMMENT 'Average pageviews per day',
        quality                         STRING        COMMENT 'Coarse evaluation of page quality',
        assess_class                    STRING        COMMENT 'WikiProject quality evaluation for the article',
        pred_class                      STRING        COMMENT 'Predicted current quality for the article',
        tag                             STRING        COMMENT 'Template type that triggered inclusion of article',
        qid                             STRING        COMMENT 'Wikidata item ID of page being recommended',
        page_id                         INT           COMMENT 'Page ID',
        gender                          ARRAY<STRING> COMMENT 'Gender value if exists and is human (as QID)',
        regions                         ARRAY<STRING> COMMENT 'List of regions associated with items'
    )
"""
print(create_table_query)
spark.sql(create_table_query)


    CREATE TABLE IF NOT EXISTS isaacj.suggestbot_recs_2021 (
        wiki_db                         STRING        COMMENT 'Wiki -- e.g., enwiki for English Wikipedia',
        user_text                       STRING        COMMENT 'User text receiving recommendations',
        revision_timestamp              STRING        COMMENT 'Revision timestamp (UTC) when recommendations were made',
        revision_id                     BIGINT        COMMENT 'Revision ID when recommendations were made',
        rank                            INT           COMMENT 'Rank of recommendation within table of 30',
        title                           STRING        COMMENT 'Recommended page title',
        views                           INT           COMMENT 'Average pageviews per day',
        quality                         STRING        COMMENT 'Coarse evaluation of page quality',
        assess_class                    STRING        COMMENT 'WikiProject quality evaluation for the article',
   

DataFrame[]

In [12]:
history_query = f"""
WITH qids AS (
  SELECT DISTINCT
    REPLACE(page_title, "_", " ") AS page_title,
    item_id
  FROM wmf.wikidata_item_page_link
  WHERE
    snapshot = '{wd_snapshot}'
    AND wiki_db = 'enwiki'
    AND page_namespace = 0
),
title_to_id AS (
    SELECT
      replace(page_title, "_", " ") AS page_title,
      page_id
    FROM wmf_raw.mediawiki_page mp
    WHERE
      snapshot = '{mw_snapshot}'
      AND wiki_db = 'enwiki'
      AND page_namespace = 0
),
recs AS (
    SELECT
      wiki_db,
      extract_user_title(page_title) AS user_text,
      revision_timestamp,
      revision_id,
      INLINE(extract_recs(revision_text))
    FROM wmf.mediawiki_wikitext_history
    WHERE
      snapshot = '{mw_snapshot}'
      AND wiki_db = '{wiki_db}'
      AND CAST(revision_timestamp AS DATE) >= '{start_date}'
      AND page_namespace = 3
      AND user_text = 'SuggestBot'
      AND revision_comment = 'SuggestBot recommends these articles...'
),
recs_with_ids AS (
    SELECT
      r.*,
      item_id,
      page_id
    FROM recs r
    LEFT JOIN qids q
      ON (r.title = q.page_title)
    LEFT JOIN title_to_id p
      ON (r.title = p.page_title)
),
regions AS (
  SELECT
    qid AS qid,
    COLLECT_SET(country) AS regions
  FROM {geo_table} g
  INNER JOIN recs_with_ids q
    ON (g.qid = q.item_id)
  WHERE
    snapshot = '{geo_snapshot}'
  GROUP BY
    qid
),
gender AS (
    SELECT
      g.item_id AS qid,
      COLLECT_SET(gender) AS gender
    FROM {gen_table} g
    INNER JOIN recs_with_ids q
      ON (g.item_id = q.item_id)
    WHERE
      snapshot = '{gen_snapshot}'
    GROUP BY
      g.item_id
)
INSERT OVERWRITE TABLE {tablename}
  SELECT
    sb.*,
    g.gender AS gender,
    r.regions AS regions
  FROM recs_with_ids sb
  LEFT JOIN gender g
    ON (sb.item_id = g.qid)
  LEFT JOIN regions r
    ON (sb.item_id = r.qid)
"""

print(history_query)
spark.sql(history_query)


WITH qids AS (
  SELECT DISTINCT
    REPLACE(page_title, "_", " ") AS page_title,
    item_id
  FROM wmf.wikidata_item_page_link
  WHERE
    snapshot = '2022-01-03'
    AND wiki_db = 'enwiki'
    AND page_namespace = 0
),
title_to_id AS (
    SELECT
      replace(page_title, "_", " ") AS page_title,
      page_id
    FROM wmf_raw.mediawiki_page mp
    WHERE
      snapshot = '2021-12'
      AND wiki_db = 'enwiki'
      AND page_namespace = 0
),
recs AS (
    SELECT
      wiki_db,
      extract_user_title(page_title) AS user_text,
      revision_timestamp,
      revision_id,
      INLINE(extract_recs(revision_text))
    FROM wmf.mediawiki_wikitext_history
    WHERE
      snapshot = '2021-12'
      AND wiki_db = 'enwiki'
      AND CAST(revision_timestamp AS DATE) >= '2021-01-01'
      AND page_namespace = 3
      AND user_text = 'SuggestBot'
      AND revision_comment = 'SuggestBot recommends these articles...'
),
recs_with_ids AS (
    SELECT
      r.*,
      item_id,
      page_id
    

DataFrame[]

## Explore data

In [13]:
spark.sql(f'select count(1), count(distinct(user_text)), count(distinct(revision_id)) from {tablename}').show(50, False)

+--------+-------------------------+---------------------------+
|count(1)|count(DISTINCT user_text)|count(DISTINCT revision_id)|
+--------+-------------------------+---------------------------+
|135390  |835                      |4513                       |
+--------+-------------------------+---------------------------+



## Move from cluster to stat machine for downloading

In [14]:
result = spark.sql(f'''
SELECT
  user_text,
  revision_timestamp,
  revision_id,
  rank,
  title,
  page_id,
  qid,
  CONCAT_WS('|', gender) AS gender,
  CONCAT_WS('|', regions) AS regions,
  views,
  quality,
  assess_class,
  pred_class,
  tag
FROM {tablename} r
ORDER BY
  user_text ASC,
  revision_timestamp ASC,
  rank ASC
''')
result.coalesce(1).write.csv(path='suggestbot-2021', compression="gzip", header=True, sep="\t")

In [15]:
!hdfs dfs -ls suggestbot-2021

Found 2 items
-rw-r-----   3 isaacj isaacj          0 2022-01-18 15:30 suggestbot-2021/_SUCCESS
-rw-r-----   3 isaacj isaacj    3805574 2022-01-18 15:30 suggestbot-2021/part-00000-9bd547b0-6922-488b-a39d-2820fe6f63ad-c000.csv.gz


In [16]:
!hdfs dfs -copyToLocal suggestbot-2021/part-00000-9bd547b0-6922-488b-a39d-2820fe6f63ad-c000.csv.gz suggestbot_2021.tsv.gz

In [17]:
!zless suggestbot_2021.tsv.gz | head -n50

user_text	revision_timestamp	revision_id	rank	title	page_id	qid	gender	regions	views	quality	assess_class	pred_class	tag
-Zai-	2021-04-13T21:49:54Z	1017640075	1	Prophets and messengers in Islam	30933488	Q168827	""	""	2166	Medium	Start	B	Add sources
-Zai-	2021-04-13T21:49:54Z	1017640075	2	Rūḥ	3022477	Q3942579	""	""	224	High	Stub	GA	Add sources
-Zai-	2021-04-13T21:49:54Z	1017640075	3	Adephaga	799626	Q131060	""	""	38	Low	C	Start	Add sources
-Zai-	2021-04-13T21:49:54Z	1017640075	4	Costa Rica	5551	Q800	""	Costa Rica	4606	Medium	B	B	Add sources
-Zai-	2021-04-13T21:49:54Z	1017640075	5	Amelia Earhart	85234	Q3355	female	United States of America	5683	Medium	B	B	Add sources
-Zai-	2021-04-13T21:49:54Z	1017640075	6	World Cat Federation	54527395	Q42713	""	""	22	Low	Start	Start	Add sources
-Zai-	2021-04-13T21:49:54Z	1017640075	7	Islam and war	30300469	Q17005933	""	""	182	Medium	Start	C	Cleanup
-Zai-	2021-04-13T21:49:54Z	1017640075	8	Cat anatomy	1519473	Q468791	""	""	584	Medium	C	B	Cleanup
-Zai-	2021-

## Testing etc.

In [11]:
static_wikitext_example = """{{Talk header preload}}
{{User:SuggestBot/config
|frequency = weekly
}}
==About me==
<div style="float:right; border=15px inset gray;">
<div style="font-size:smaller; text-align:center;">Education</div>
{{User Penn}}<br />
{{User Villanova}}<br />
</div>
'''Wiki-psyc''' (logged out IP 97.99.90.245)
:'''Occupation:''' Medical, Life sciences<br>
:'''Other interests:''' [[Wikipedia:WikiProject Psychology/Popular pages|Psychology]], Philosophy, Technology 
:'''Editing objectives:''' 
::* Improving usefulness - writing in clear and understandable language for the reader
::* Improving cohesion - organizing sentences, paragraphs, sections together to so that the overall message is clear/logical to the reader.
::* Improving accuracy - prioritizing content that represents the state-of-the-art and/or expert consensus
:'''Age:''' 67
<hr>

== Articles you might like to edit, from SuggestBot ==
''Note:'' All columns in this table are sortable, allowing you to rearrange the table so the articles most interesting to you are shown at the top.  All images have mouse-over popups with more information.  For more information about the columns and categories, please [[User:SuggestBot/Documentation/Suggestion columns|consult the documentation]] and please get in touch on [[User talk:SuggestBot|SuggestBot's talk page]] with any questions you might have.

{| class="wikitable sortable" border="1" style="text-align: center;"
|-
! scope="col" | Views/Day
! scope="col" | Quality
! scope="col" | Title
! scope="col" | Tagged with…

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:526}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Pathological lying|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Pathological lying]] <small>([[Talk:Pathological lying|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:142}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Employee assistance program|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Employee assistance program]] <small>([[Talk:Employee assistance program|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:50}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Astrojax|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Astrojax]] <small>([[Talk:Astrojax|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:109}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: C, Predicted class: FA|link=Haltlose personality disorder|Quality: High, Assessed class: C, Predicted class: FA]]
| style="text-align: left;" | [[Haltlose personality disorder]] <small>([[Talk:Haltlose personality disorder|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:3132}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: C, Predicted class: FA|link=Emotion|Quality: High, Assessed class: C, Predicted class: FA]]
| style="text-align: left;" | [[Emotion]] <small>([[Talk:Emotion|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:173}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=V.Smile|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[V.Smile]] <small>([[Talk:V.Smile|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:524}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: C, Predicted class: FA|link=Abusive power and control|Quality: High, Assessed class: C, Predicted class: FA]]
| style="text-align: left;" | [[Abusive power and control]] <small>([[Talk:Abusive power and control|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:40}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: B|link=Cyberattacks during the Russo-Georgian War|Quality: Medium, Assessed class: C, Predicted class: B]]
| style="text-align: left;" | [[Cyberattacks during the Russo-Georgian War]] <small>([[Talk:Cyberattacks during the Russo-Georgian War|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:22}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Flagging dance|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Flagging dance]] <small>([[Talk:Flagging dance|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:44}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: B|link=Paid news in India|Quality: Medium, Assessed class: Start, Predicted class: B]]
| style="text-align: left;" | [[Paid news in India]] <small>([[Talk:Paid news in India|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:146}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: NA, Predicted class: Start|link=Satoru Nomura|Quality: Low, Assessed class: NA, Predicted class: Start]]
| style="text-align: left;" | [[Satoru Nomura]] <small>([[Talk:Satoru Nomura|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:66}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Start|link=Personality disorder not otherwise specified|Quality: Low, Assessed class: Start, Predicted class: Start]]
| style="text-align: left;" | [[Personality disorder not otherwise specified]] <small>([[Talk:Personality disorder not otherwise specified|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:12185}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: B|link=Seven deadly sins|Quality: Medium, Assessed class: C, Predicted class: B]]
| style="text-align: left;" | [[Seven deadly sins]] <small>([[Talk:Seven deadly sins|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:62}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Narcissistic mortification|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Narcissistic mortification]] <small>([[Talk:Narcissistic mortification|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:74}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Fan dance|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Fan dance]] <small>([[Talk:Fan dance|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:486}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Malignant narcissism|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Malignant narcissism]] <small>([[Talk:Malignant narcissism|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:5777}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: B|link=Social media|Quality: Medium, Assessed class: C, Predicted class: B]]
| style="text-align: left;" | [[Social media]] <small>([[Talk:Social media|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:8}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: NA, Predicted class: C|link=Comparison of Object Pascal and C|Quality: Medium, Assessed class: NA, Predicted class: C]]
| style="text-align: left;" | [[Comparison of Object Pascal and C]] <small>([[Talk:Comparison of Object Pascal and C|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:73}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Theodore Millon|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Theodore Millon]] <small>([[Talk:Theodore Millon|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:454}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Conduct disorder|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Conduct disorder]] <small>([[Talk:Conduct disorder|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:1064}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: C, Predicted class: GA|link=Whataboutism|Quality: High, Assessed class: C, Predicted class: GA]]
| style="text-align: left;" | [[Whataboutism]] <small>([[Talk:Whataboutism|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:11}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: NA, Predicted class: C|link=Fauxtography|Quality: Medium, Assessed class: NA, Predicted class: C]]
| style="text-align: left;" | [[Fauxtography]] <small>([[Talk:Fauxtography|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:11}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Trauma-informed feminist therapy|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Trauma-informed feminist therapy]] <small>([[Talk:Trauma-informed feminist therapy|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:35}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Start|link=Ross W. Greene|Quality: Low, Assessed class: Start, Predicted class: Start]]
| style="text-align: left;" | [[Ross W. Greene]] <small>([[Talk:Ross W. Greene|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:196}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Start|link=Semantic Scholar|Quality: Low, Assessed class: Start, Predicted class: Start]]
| style="text-align: left;" | [[Semantic Scholar]] <small>([[Talk:Semantic Scholar|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:15}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Agent Communications Language|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Agent Communications Language]] <small>([[Talk:Agent Communications Language|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:12}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Meteor (juggling)|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Meteor (juggling)]] <small>([[Talk:Meteor (juggling)|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:366}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Messiah complex|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Messiah complex]] <small>([[Talk:Messiah complex|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:22}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Leandro Firmino|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Leandro Firmino]] <small>([[Talk:Leandro Firmino|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:9}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: NA, Predicted class: Start|link=Rubenstein's Revenge|Quality: Low, Assessed class: NA, Predicted class: Start]]
| style="text-align: left;" | [[Rubenstein's Revenge]] <small>([[Talk:Rubenstein's Revenge|talk]])</small>
| Stub
|}

SuggestBot picks articles in a number of ways based on other articles you've edited, including straight text similarity, following wikilinks, and matching your editing patterns against those of other Wikipedians.  It tries to recommend only articles that other Wikipedians have marked as needing work.  We appreciate that you have signed up to receive suggestions regularly; your contributions make Wikipedia better — thanks for helping!

If you have feedback on how to make SuggestBot better, please let us know on [[User_talk:SuggestBot|SuggestBot's talk page]]. -- [[User:SuggestBot|SuggestBot]] ([[User talk:SuggestBot|talk]]) 23:25, 31 October 2021 (UTC)

== November 2021 ==

[[File:Information.svg|25px|alt=Information icon]] Hi, and thank you for [[Special:Contributions/Wiki-psyc|your contributions]] to Wikipedia. It appears that you tried to give a page a different title by copying its content and pasting either the same content, or an edited version of it, into [[:Megalomania]]. This is known as a "[[Wikipedia:Administrators' guide/Fixing cut-and-paste moves|cut-and-paste move]]", and it is undesirable because it splits the [[Help:Page history|page history]], which is [[Wikipedia:Copying within Wikipedia|legally required for attribution]].  Instead, the software used by Wikipedia has a feature that allows pages to be ''moved'' to a new title together with their edit history.

In most cases for registered users, once your account is [[Wikipedia:Autoconfirmed|four days old and has ten edits]], you should be able to move an article yourself using the [[Help:Moving a page|"Move" tab]] at the top of the page (the tab may be [[:File:Vector hidden move button.png|hidden in a dropdown menu]] for you). This both preserves the page history intact and automatically creates a [[Wikipedia:Redirect|redirect]] from the old title to the new. If you cannot perform a particular page move yourself this way (e.g. because a page already exists at the target title), please follow the instructions at [[Wikipedia:Requested moves|requested moves]] to have it moved by someone else. Also, if there are any other pages that you moved by copying and pasting, even if it was a long time ago, please list them at [[Wikipedia:Requests for history merge]]. ''<p>
Please seek [[Wikipedia:Consensus|consensus]] in the [[WP:RM#CM|Requested moves]] process for these kind of [[Wikipedia:Disambiguation|disambiguation]] situations.</p>'' <!-- Template:uw-c&pmove --> [[User:LittleWink|LittleWink]] ([[User talk:LittleWink|talk]]) 20:02, 1 November 2021 (UTC)"""

# test
extract_recs(static_wikitext_example)

['1|Pathological lying|526|Medium|Start|C|Add sources',
 '2|Employee assistance program|142|Medium|Start|C|Add sources',
 '3|Astrojax|50|Medium|Start|C|Add sources',
 '4|Haltlose personality disorder|109|High|C|FA|Add sources',
 '5|Emotion|3132|High|C|FA|Add sources',
 '6|V.Smile|173|Medium|Start|C|Add sources',
 '7|Abusive power and control|524|High|C|FA|Cleanup',
 '8|Cyberattacks during the Russo-Georgian War|40|Medium|C|B|Cleanup',
 '9|Flagging dance|22|Medium|Start|C|Cleanup',
 '10|Paid news in India|44|Medium|Start|B|Expand',
 '11|Satoru Nomura|146|Low|NA|Start|Expand',
 '12|Personality disorder not otherwise specified|66|Low|Start|Start|Expand',
 '13|Seven deadly sins|12185|Medium|C|B|Unencyclopaedic',
 '14|Narcissistic mortification|62|Medium|C|C|Unencyclopaedic',
 '15|Fan dance|74|Low|Stub|Start|Unencyclopaedic',
 '16|Malignant narcissism|486|Medium|C|C|Merge',
 '17|Social media|5777|Medium|C|B|Merge',
 '18|Comparison of Object Pascal and C|8|Medium|NA|C|Merge',
 '19|Theodore M

In [15]:
spark.sql('select extract_user_title("User_talk:I_Am_Chaos/current")').show(50, False)

+------------------------------------------------+
|extract_user_title(User_talk:I_Am_Chaos/current)|
+------------------------------------------------+
|I_Am_Chaos                                      |
+------------------------------------------------+



In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame([wt], columns=['wikitext'])
spark.createDataFrame(df).createOrReplaceTempView('test_data')

In [11]:
spark.sql('select inline(extract_recs(wikitext)) from test_data').show(50, False)

+----+---------------------------------------------------------------+-----+-------+------------+----------+---------------+
|rank|title                                                          |views|quality|assess_class|pred_class|tag            |
+----+---------------------------------------------------------------+-----+-------+------------+----------+---------------+
|1   |Greco-Buddhist monasticism                                     |37   |Medium |Start       |B         |Add sources    |
|2   |Nikonion                                                       |18   |Medium |Stub        |C         |Add sources    |
|3   |Center for Infectious Disease Research and Policy              |17   |Medium |Stub        |C         |Add sources    |
|4   |Tick-borne encephalitis                                        |207  |High   |Start       |GA        |Add sources    |
|5   |Block matrix                                                   |390  |Medium |Start       |C         |Add sources    |


In [7]:
wt = """== Welcome! ==
{| style="background-color:#F5FFFA; padding:0;"
|class="MainPageBG" style="border:1px solid lightgrey; background-color:lightgrey; vertical-align:top; color:#000000; font-size:85%"|
{| style="width: 100%; padding: 0px; vertical-align:top; background-color:lavender; padding:0;"
! <div style="margin:0; background-color:white; font-family:sans-serif; font-size:120%; font-weight:normal; border:1px solid lightgrey; text-align:left; padding-left:0.4em; padding-top:0.2em; padding-bottom:0.2em;">Hello, Gnittopsniart! '''[[Wikipedia:Introduction|Welcome]] to Wikipedia!''' Thank you for [[Special:Contributions/Gnittopsniart|your contributions]]. You may benefit from following some of the links below, which will help you get the most out of Wikipedia. If you have any questions you can ask me on my talk page, or place '''<code>{&#123;helpme&#125;}</code>''' on your [[Help:Talk pages|talk page]] and ask your question there. Please remember to [[Wikipedia:Signatures|sign your name]] on talk pages by clicking [[File:OOUI JS signature icon LTR.png|22px|link=Wikipedia:How to sign your posts|alt=]] or by typing four tildes "~&#126;~~"; this will automatically produce your name and the date. If you are already excited about Wikipedia,  you might want to consider being "[[Wikipedia:Adopt-a-user|adopted]]" by a more experienced editor or joining a [[Wikipedia:WikiProject|WikiProject]] to collaborate with others in creating and improving articles of your interest. Click [[Wikipedia:WikiProject Council/Directory|here]] for a directory of all the WikiProjects. Finally, please do your best to always fill in the [[Help:Edit summary|edit summary]] field when making edits to pages. Happy editing! [[User:Paul2520|paul2520]] ([[User talk:Paul2520|talk]]) 14:57, 23 March 2020 (UTC)</div>
|}
{| width="100%" style="background-color:white;"
|class="MainPageBG" style="width: 55%; border:1px solid white; background-color:lightgrey; vertical-align:top"|
{| style="width: 100%; padding: 0px; vertical-align:top; background-color:Lavender"
! <div style="margin: 0; background-color:white; font-family: sans-serif; font-size:120%; font-weight:bold; border:1px solid lightgrey; text-align:left; color:grey; padding-left:0.4em; padding-top: 0.2em; padding-bottom: 0.2em;">Getting Started</div>
|-
|style="color:#000"|
* [[Wikipedia:Contributing to Wikipedia|Contributing to Wikipedia]] • [[Wikipedia:Tutorial|Tutorial]] • [[Wikipedia:Your first article|Your first article]]
* [[Wikipedia:Five pillars|The five pillars of Wikipedia]] • [[Wikipedia:Simplified ruleset|Simplified ruleset]]
* How to: [[Help:Editing|edit a page]] • [[Wikipedia:Uploading images|upload an image]]
|-
! <div style="margin: 0; background:white; font-family: sans-serif; font-size:120%; font-weight:bold; border:1px solid lightgrey; text-align:left; color:grey; padding-left:0.4em; padding-top: 0.2em; padding-bottom: 0.2em;">Getting Help</div>
|-
| style="color:#000"|
* [[Wikipedia:FAQ|Frequently Asked Questions]]
*[[Wikipedia:Directories and indexes|Directories and indexes]]
* Where to: [[Wikipedia:Questions|ask questions]] • [[Wikipedia:Request directory|make requests]]
* [[Wikipedia:Tip of the day|Tip of the day]]
* [[Wikipedia:Requests for administrator attention|Request administrator attention]]
|-
! <div style="margin: 0; background:white; font-family: sans-serif; font-size:120%; font-weight:bold; border:1px solid lightgrey; text-align:left; color:grey; padding-left:0.4em; padding-top: 0.2em; padding-bottom: 0.2em;">Policies and Guidelines</div>
|-
| style="color:#000"|
* [[Wikipedia:Neutral point of view|Neutral point of view]] • [[Wikipedia:Identifying reliable sources|Reliable sources]]
* [[Wikipedia:Verifiability|Verifiability]] • [[Help:Introduction to referencing with Wiki Markup/1|Citing sources]] • [[Wikipedia:No original research|No original research]]
* [[Wikipedia:What Wikipedia is not|What Wikipedia is not]] • [[Wikipedia:Biographies of living persons|Biographies of living persons]]
----
* [[Wikipedia:Manual of Style|Manual of Style]] ([[Wikipedia:Simplified Manual of Style|Simplified]]) • [[Wikipedia:Edit warring#The three-revert rule|Three-revert rule]]
* [[Wikipedia:Copyrights|Copyrights]] • [[Wikipedia:Non-free content|Non-free content]] • [[Wikipedia:Image use policy|Image use policy]]
* [[Wikipedia:External links|External links]] • [[Wikipedia:Spam|Spam]] • [[Wikipedia:Vandalism|Vandalism]] • [[Wikipedia:Sock puppetry|Sock puppetry]]
* [[Wikipedia:Deletion policy|Deletion policy]] • [[Wikipedia:Conflict of interest|Conflict of interest]] • [[Wikipedia:Notability|Notability]]
|-
|}
|class="MainPageBG" style="width: 55%; border:1px solid lightgrey; background-color:Seashell; vertical-align:top"|
{| style="width: 100%; padding: 0px; vertical-align:top; background-color:Seashell"
! <div style="margin: 0; background-color:white; font-family: sans-serif; font-size:120%; font-weight:bold; border:1px solid lightgrey; text-align:left; color:grey; padding-left:0.4em; padding-top: 0.2em; padding-bottom: 0.2em;">The Community</div>
|-
|style="color:#000"|
* [[Wikipedia:Community portal|Community Portal]]  • [[Wikipedia:Adopt-a-user|Adopt-a-user program]] 
* [[Wikipedia:Assume good faith|Assume good faith]] • [[Wikipedia:Civility|Civility]] • [[Wikipedia:Etiquette|Etiquette]]
* [[Wikipedia:No personal attacks|No personal attacks]] • [[Wikipedia:Dispute resolution|Resolving disputes]]
* [[Wikipedia:Consensus|Build consensus]] • [[Wikipedia:Village pump|Village pump]]
* [[Wikipedia:IRC|IRC channels]] • [[Wikipedia:Mailing lists|Mailing lists]]
* [[Wikipedia:Wikipedia Signpost|''The Signpost'' (Wikipedia's newspaper)]]
|-
! <div style="margin: 0; background-color:white; font-family: sans-serif; font-size:120%; font-weight:bold; border:1px solid lightgrey; text-align:left; color:grey; padding-left:0.4em; padding-top: 0.2em; padding-bottom: 0.2em;">Things to do</div>
|-
|style="color:#000"|
* '''[[Wikipedia:Be bold|Be bold in editing]]''' • [[Wikipedia:Article development|Help develop an article]]
* [[Wikipedia:Maintenance|Maintenance]] • [[Wikipedia:Peer review|Peer review]] • [[Wikipedia:Requested articles|Requested articles]]
* [[Wikipedia:WikiProject|Join a WikiProject]] • [[Wikipedia:Translation|Translating articles]]
* Cleaning up: [[Wikipedia:Cleanup|General]] • [[Wikipedia:WikiProject Spam|Spam]] • [[Wikipedia:Cleaning up vandalism|Vandalism]]
|-
! <div style="margin: 0; background-color:white; font-family: sans-serif; font-size:120%; font-weight:bold; border:1px solid lightgrey; text-align:left; color:grey; padding-left:0.4em; padding-top: 0.2em; padding-bottom: 0.2em;">Miscellaneous</div>
|-
|style="color:#000"|
* [[Wikipedia:Categorization|Categories]] • [[Wikipedia:Disambiguation|Disambiguity]] • [[Wikipedia:Stub|Stubs]]
* [[Wikipedia:User pages|User pages]] • [[Help:Talk pages|Talk pages]]
* [[Wikipedia:Template messages|Useful templates]] • [[Wikipedia:Tools|Tools]] • [[Wikipedia:User scripts|User scripts]]
|}
|}
|}<!--Template:WelcomeMenu-->
== Nomination of [[:DIVE Studios]] for deletion ==
<div class="floatleft" style="margin-bottom:0">[[File:Ambox warning orange.svg|48px|alt=|link=]]</div>A discussion is taking place as to whether the article '''[[:DIVE Studios]]''' is suitable for inclusion in Wikipedia according to [[Wikipedia:List of policies and guidelines|Wikipedia's policies and guidelines]] or whether it should be [[Wikipedia:Deletion policy|deleted]].

The article will be discussed at [[Wikipedia:Articles for deletion/DIVE Studios]] until a consensus is reached, and anyone, including you, is welcome to contribute to the discussion. The nomination will explain the policies and guidelines which are of concern. The discussion focuses on high-quality evidence and our policies and guidelines.

Users may edit the article during the discussion, including to improve the article to address concerns raised in the discussion. However, do not remove the article-for-deletion notice from the top of the article.<!-- Template:afd notice --> [[User:Thebiv19|Thebiv19]] ([[User talk:Thebiv19|talk]]) 14:28, 26 April 2020 (UTC)

== Your submission at [[WP:AFC|Articles for creation]] ==

<div style="border: solid 1px #FCC; background-color: #F8EEBC; padding: 0.5em 1em; color: #000; margin: 1.5em; width: 90%;"> [[File:AFC-Logo_Decline.svg|50px|left]]Your recent article submission to [[Wikipedia:Articles for creation|Articles for Creation]] has been reviewed! Unfortunately, it has not been accepted at this time.<!--
--
--> Please check the submission for any additional comments left by the reviewer.  You are encouraged to edit the submission to address the issues raised and resubmit ''when they have been resolved''.
{{clear}}
* If you would like to continue working on the submission, go to [[Draft:Andrea Ammon]] and click on the "Edit" tab at the top of the window.
* If you now believe the draft cannot meet Wikipedia's standards or do not wish to progress it further, you may request deletion. Please go to [[Draft:Andrea Ammon]], click on the "Edit" tab at the top of the window, add "{{tl|Db-g7}}" at the top of the draft text and click the blue "publish changes" button to save this edit.
* If you do not make any further changes to your draft, in 6 months, it will be considered abandoned and [[Wikipedia:Criteria for speedy deletion#G13. Abandoned Drafts and Articles for creation submissions|may be deleted]].
* If you need any assistance, you can ask for help at the <span class="plainlinks">[//en.wikipedia.org/w/index.php?title=Wikipedia:WikiProject_Articles_for_creation/Help_desk&action=edit&section=new&nosummary=1&preload=Template:Afc_decline/HD_preload&preloadparams%5B%5D=Draft:Andrea_Ammon '''Articles for creation help desk''']</span>, on the <span class="plainlinks">[//en.wikipedia.org/w/index.php?title=User_talk:1292simon&action=edit&section=new&nosummary=1&preload=Template:Afc_decline/HD_preload&preloadparams%5B%5D=Draft:Andrea_Ammon '''reviewer's talk page''']</span> or use [[Wikipedia:IRC help disclaimer|Wikipedia's real-time chat help from experienced editors]].

[[User:1292simon|1292simon]] ([[User talk:1292simon|talk]]) 09:22, 10 May 2020 (UTC)</div><!--Template:Afc decline-->
==Concern regarding [[Draft:Andrea Ammon]]==
[[File:Information.svg|25px|alt=Information icon]] Hello, Gnittopsniart. This is a bot-delivered message letting you know that [[Draft:Andrea Ammon]], a page you created, has not been edited in at least 5 months.  Draft space is not an indefinite storage location for content that is not appropriate for [[WP:MAINSPACE|article space]].

If your submission is not edited soon, it could be nominated for deletion under [[WP:G13|CSD G13]].  If you would like to attempt to save it, you will need to improve it. You may request [[WP:USERFY|userfication]] of the content if it meets requirements.

If the deletion has already occured, instructions on how you may be able to retrieve it are available [[WP:REFUND/G13|here]].

Thank you for your submission to Wikipedia. [[User:FireflyBot|FireflyBot]] ([[User talk:FireflyBot|talk]]) 07:04, 6 April 2021 (UTC)

== Your submission at [[Wikipedia:Articles for creation|Articles for creation]]: [[Ines Steffens]] has been accepted ==
<div style="border:solid 1px #57DB1E; background:#E6FFE6; padding:1em; padding-top:0.5em; padding-bottom:0.5em; width:20em; color:black; margin-bottom: 1.5em; margin-left: 1.5em; width: 90%;">[[File:AFC-Logo.svg|50px|left]] '''[[Ines Steffens]], which you submitted to Articles for creation, has been created.'''<br />

Congratulations, and thank you for helping expand the scope of Wikipedia! We hope you will continue making quality contributions. 

<div class="autoconfirmed-show">Since you have made at least 10 edits over more than four days, you can now [[Wikipedia:Your first article|create articles yourself]] without posting a request. However, you may continue submitting work to [[Wikipedia:Articles for creation|Articles for creation]] if you prefer.</div>

If you have any questions, you are welcome to ask at the '''<span class="plainlinks">[//en.wikipedia.org/w/index.php?title=Wikipedia:WikiProject_Articles_for_creation/Help_desk&action=edit&section=new&nosummary=1&preload=Template:AfC_talk/HD_preload&preloadparams%5B%5D=Ines_Steffens help desk]</span>'''.<span class="unconfirmed-show"> Once you have made at least 10 edits and had an account for at least four days, you will have the option to [[Wikipedia:Your first article|create articles yourself]] without posting a request to [[Wikipedia:Articles for creation|Articles for creation]].</span>

If you would like to help us improve this process, please consider {{leave feedback/link|page=Wikipedia:WikiProject Articles for creation|text=leaving us some feedback}}.

Thanks again, and happy editing!
[[User:WikiAviator|<span style="colour:#005eb8;">'''WikiAviator'''</span>]]<sup>[[User talk:WikiAviator|'' talk'']]</sup> 07:10, 23 April 2021 (UTC)</div><!--Template:Afc talk-->

== Your submission at [[Wikipedia:Articles for creation|Articles for creation]]: [[Detention of Alek Sigley]] has been accepted ==
<div style="border:solid 1px #57DB1E; background:#E6FFE6; padding:1em; padding-top:0.5em; padding-bottom:0.5em; width:20em; color:black; margin-bottom: 1.5em; margin-left: 1.5em; width: 90%;">[[File:AFC-Logo.svg|50px|left]] '''[[Detention of Alek Sigley]], which you submitted to Articles for creation, has been created.'''<br />

Congratulations, and thank you for helping expand the scope of Wikipedia! We hope you will continue making quality contributions. <br />

The article has been assessed as '''Start-Class''', which is recorded on its [[Talk:Detention of Alek Sigley|talk page]]. Most new articles start out as Stub-Class or Start-Class and then attain higher grades as they [[Wikipedia:Article development|develop]] over time. You may like to take a look at the [[Wikipedia:WikiProject Articles for creation/Grading scheme|grading scheme]] to see how you can improve the article.

<div class="autoconfirmed-show">Since you have made at least 10 edits over more than four days, you can now [[Wikipedia:Your first article|create articles yourself]] without posting a request. However, you may continue submitting work to [[Wikipedia:Articles for creation|Articles for creation]] if you prefer.</div>

If you have any questions, you are welcome to ask at the '''<span class="plainlinks">[//en.wikipedia.org/w/index.php?title=Wikipedia:WikiProject_Articles_for_creation/Help_desk&action=edit&section=new&nosummary=1&preload=Template:AfC_talk/HD_preload&preloadparams%5B%5D=Detention_of_Alek_Sigley help desk]</span>'''.<span class="unconfirmed-show"> Once you have made at least 10 edits and had an account for at least four days, you will have the option to [[Wikipedia:Your first article|create articles yourself]] without posting a request to [[Wikipedia:Articles for creation|Articles for creation]].</span>

If you would like to help us improve this process, please consider {{leave feedback/link|page=Wikipedia:WikiProject Articles for creation|text=leaving us some feedback}}.

Thanks again, and happy editing!
[[User:CaptainEek|<span style="color:#6a1f7f">'''CaptainEek'''</span>]] <sup>[[User talk:CaptainEek|<span style="font-size:82%"><span style="color:#a479e5">''Edits Ho Cap'n!''</span></span>]]</sup>[[Special:Contributions/CaptainEek|⚓]] 01:25, 17 May 2021 (UTC)</div><!--Template:Afc talk-->

== Articles you might like to edit, from SuggestBot ==
''Note:'' All columns in this table are sortable, allowing you to rearrange the table so the articles most interesting to you are shown at the top.  All images have mouse-over popups with more information.  For more information about the columns and categories, please [[User:SuggestBot/Documentation/Suggestion columns|consult the documentation]] and please get in touch on [[User talk:SuggestBot|SuggestBot's talk page]] with any questions you might have.

{| class="wikitable sortable" border="1" style="text-align: center;"
|-
! scope="col" | Views/Day
! scope="col" | Quality
! scope="col" | Title
! scope="col" | Tagged with…

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:447}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: Start, Predicted class: GA|link=Tick-borne encephalitis|Quality: High, Assessed class: Start, Predicted class: GA]]
| style="text-align: left;" | [[Tick-borne encephalitis]] <small>([[Talk:Tick-borne encephalitis|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:31}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: B|link=Greco-Buddhist monasticism|Quality: Medium, Assessed class: Start, Predicted class: B]]
| style="text-align: left;" | [[Greco-Buddhist monasticism]] <small>([[Talk:Greco-Buddhist monasticism|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:2390}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: B|link=Tedros Adhanom|Quality: Medium, Assessed class: C, Predicted class: B]]
| style="text-align: left;" | [[Tedros Adhanom]] <small>([[Talk:Tedros Adhanom|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:19}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Stub, Predicted class: C|link=Center for Infectious Disease Research and Policy|Quality: Medium, Assessed class: Stub, Predicted class: C]]
| style="text-align: left;" | [[Center for Infectious Disease Research and Policy]] <small>([[Talk:Center for Infectious Disease Research and Policy|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:14}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: NA, Predicted class: C|link=Field epidemiology|Quality: Medium, Assessed class: NA, Predicted class: C]]
| style="text-align: left;" | [[Field epidemiology]] <small>([[Talk:Field epidemiology|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:20}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Stub, Predicted class: C|link=Nikonion|Quality: Medium, Assessed class: Stub, Predicted class: C]]
| style="text-align: left;" | [[Nikonion]] <small>([[Talk:Nikonion|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:105}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Athenian festivals|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Athenian festivals]] <small>([[Talk:Athenian festivals|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:171}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Regions of ancient Greece|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Regions of ancient Greece]] <small>([[Talk:Regions of ancient Greece|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:236}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Doric Greek|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Doric Greek]] <small>([[Talk:Doric Greek|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:118}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Stub, Predicted class: C|link=Robert Koch Institute|Quality: Medium, Assessed class: Stub, Predicted class: C]]
| style="text-align: left;" | [[Robert Koch Institute]] <small>([[Talk:Robert Koch Institute|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:24}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: NA, Predicted class: C|link=Caulonia (ancient city)|Quality: Medium, Assessed class: NA, Predicted class: C]]
| style="text-align: left;" | [[Caulonia (ancient city)]] <small>([[Talk:Caulonia (ancient city)|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:22}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Aljaraque|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Aljaraque]] <small>([[Talk:Aljaraque|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:153}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Giresun|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Giresun]] <small>([[Talk:Giresun|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:74}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: B|link=The Wasps|Quality: Medium, Assessed class: Start, Predicted class: B]]
| style="text-align: left;" | [[The Wasps]] <small>([[Talk:The Wasps|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:215}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: B, Predicted class: C|link=Prostitution in ancient Greece|Quality: Medium, Assessed class: B, Predicted class: C]]
| style="text-align: left;" | [[Prostitution in ancient Greece]] <small>([[Talk:Prostitution in ancient Greece|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:91}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: Start, Predicted class: GA|link=Lives and Opinions of Eminent Philosophers|Quality: High, Assessed class: Start, Predicted class: GA]]
| style="text-align: left;" | [[Lives and Opinions of Eminent Philosophers]] <small>([[Talk:Lives and Opinions of Eminent Philosophers|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:240}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: Start, Predicted class: GA|link=Diogenes Laërtius|Quality: High, Assessed class: Start, Predicted class: GA]]
| style="text-align: left;" | [[Diogenes Laërtius]] <small>([[Talk:Diogenes Laërtius|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:172}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: NA, Predicted class: B|link=26th Critics' Choice Awards|Quality: Medium, Assessed class: NA, Predicted class: B]]
| style="text-align: left;" | [[26th Critics' Choice Awards]] <small>([[Talk:26th Critics' Choice Awards|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:10}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: C, Predicted class: GA|link=2013–2014 Zika virus outbreaks in Oceania|Quality: High, Assessed class: C, Predicted class: GA]]
| style="text-align: left;" | [[2013–2014 Zika virus outbreaks in Oceania]] <small>([[Talk:2013–2014 Zika virus outbreaks in Oceania|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:5}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Abraham Lilienfeld|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Abraham Lilienfeld]] <small>([[Talk:Abraham Lilienfeld|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:7}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: NA, Predicted class: Start|link=Improvement Science Research Network|Quality: Low, Assessed class: NA, Predicted class: Start]]
| style="text-align: left;" | [[Improvement Science Research Network]] <small>([[Talk:Improvement Science Research Network|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:6}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Start|link=Omar Abdi|Quality: Low, Assessed class: Start, Predicted class: Start]]
| style="text-align: left;" | [[Omar Abdi]] <small>([[Talk:Omar Abdi|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:3}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: NA, Predicted class: B|link=Research Centre for Emerging and Reemerging Infectious Diseases|Quality: Medium, Assessed class: NA, Predicted class: B]]
| style="text-align: left;" | [[Research Centre for Emerging and Reemerging Infectious Diseases]] <small>([[Talk:Research Centre for Emerging and Reemerging Infectious Diseases|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:2}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Carole Presern|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Carole Presern]] <small>([[Talk:Carole Presern|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:346}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=People Just Do Nothing: Big in Japan|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[People Just Do Nothing: Big in Japan]] <small>([[Talk:People Just Do Nothing: Big in Japan|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:18}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Stub|link=Kepoi|Quality: Low, Assessed class: Start, Predicted class: Stub]]
| style="text-align: left;" | [[Kepoi]] <small>([[Talk:Kepoi|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:22}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: C, Predicted class: FA|link=Entomophthora|Quality: High, Assessed class: C, Predicted class: FA]]
| style="text-align: left;" | [[Entomophthora]] <small>([[Talk:Entomophthora|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:690}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Allan Mustafa|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Allan Mustafa]] <small>([[Talk:Allan Mustafa|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:3}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Stub|link=Annals of Clinical & Laboratory Science|Quality: Low, Assessed class: Stub, Predicted class: Stub]]
| style="text-align: left;" | [[Annals of Clinical & Laboratory Science]] <small>([[Talk:Annals of Clinical & Laboratory Science|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:8}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Clinical Microbiology and Infection|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Clinical Microbiology and Infection]] <small>([[Talk:Clinical Microbiology and Infection|talk]])</small>
| Stub
|}

SuggestBot picks articles in a number of ways based on other articles you've edited, including straight text similarity, following wikilinks, and matching your editing patterns against those of other Wikipedians.  It tries to recommend only articles that other Wikipedians have marked as needing work.  We appreciate that you have signed up to receive suggestions regularly; your contributions make Wikipedia better — thanks for helping!

If you have feedback on how to make SuggestBot better, please let us know on [[User_talk:SuggestBot|SuggestBot's talk page]]. -- [[User:SuggestBot|SuggestBot]] ([[User talk:SuggestBot|talk]]) 11:23, 19 May 2021 (UTC)

== Your submission at [[Wikipedia:Articles for creation|Articles for creation]]: [[Draft:Masaji Ishikawa|Masaji Ishikawa]] (May 24) ==
<div style="border: solid 1px #FCC; background-color: #F8EEBC; padding: 0.5em 1em; color: #000; margin: 1.5em; width: 90%;"> [[File:AFC-Logo_Decline.svg|50px|left]]Your recent article submission to [[Wikipedia:Articles for creation|Articles for Creation]] has been reviewed! Unfortunately, it has not been accepted at this time.<nowiki> </nowiki>The reason left by Pbrks was:

{{divbox|gray|3=This submission provides insufficient [[Wikt:context|context]] for those unfamiliar with the subject matter. Please see the [[Wikipedia:Guide to writing better articles|guide to writing better articles]] for information on how to better format your submission.|}}<!--
--
-->&nbsp;The comment the reviewer left was:

{{divbox|blue|3=It is very unclear who this person is. The first sentence should tell a nonspecialist reader what, or who, the subject is. See [[MOS:FIRST]] for more information.}} Please check the submission for any additional comments left by the reviewer.  You are encouraged to edit the submission to address the issues raised and resubmit ''when they have been resolved''.
{{clear}}
* If you would like to continue working on the submission, go to [[Draft:Masaji Ishikawa]] and click on the "Edit" tab at the top of the window.
* If you now believe the draft cannot meet Wikipedia's standards or do not wish to progress it further, you may request deletion. Please go to [[Draft:Masaji Ishikawa]], click on the "Edit" tab at the top of the window, add "{{tl|Db-g7}}" at the top of the draft text and click the blue "publish changes" button to save this edit.
* If you do not make any further changes to your draft, in 6 months, it will be considered abandoned and [[Wikipedia:Criteria for speedy deletion#G13. Abandoned Drafts and Articles for creation submissions|may be deleted]].
* If you need any assistance, or have experienced any [[Wikipedia:Articles for creation/Scam warning|untoward behavior]] associated with this submission, you can ask for help at the <span class="plainlinks">[//en.wikipedia.org/w/index.php?title=Wikipedia:WikiProject_Articles_for_creation/Help_desk&action=edit&section=new&nosummary=1&preload=Template:AfC_decline/HD_preload&preloadparams%5B%5D=Draft:Masaji_Ishikawa '''Articles for creation help desk''']</span>, on the <span class="plainlinks">[//en.wikipedia.org/w/index.php?title=User_talk:Pbrks&action=edit&section=new&nosummary=1&preload=Template:AfC_decline/HD_preload&preloadparams%5B%5D=Draft:Masaji_Ishikawa '''reviewer's talk page''']</span> or use [[Wikipedia:IRC help disclaimer|Wikipedia's real-time chat help from experienced editors]].

[[User:Pbrks|Pbrks]] ([[User talk:Pbrks|talk]]) 04:38, 24 May 2021 (UTC)</div><!--Template:AfC decline-->

{| style="margin: 0.4em 2em;"
|- style="vertical-align: top;"
| [[File:WP teahouse logo 2.png|alt=Teahouse logo]]
| <div style="background-color:#e1e6db; color: #393D38; padding: 1em; font-size: 1.1em; border-radius:10px;box-shadow:-2px -2px 1px #8e8a78;">Hello, '''Gnittopsniart'''!
Having an article declined at Articles for Creation can be disappointing. If you are wondering why your article submission was declined, please post a question at the '''[[Wikipedia:WikiProject Articles for creation/Help desk|Articles for creation help desk]]'''. If you have any ''other'' questions about your editing experience, we'd love to help you at the '''[[Wikipedia:Teahouse|Teahouse]]''', a friendly space on Wikipedia where experienced editors lend a hand to help new editors like yourself! See you there! [[User:Pbrks|Pbrks]] ([[User talk:Pbrks|talk]]) 04:38, 24 May 2021 (UTC)</div>
|}<!-- Wikipedia:Teahouse/AfC Invitation -->
[[Category:Wikipedians who have received a Teahouse invitation through AfC]]

==Disambiguation link notification for May 24==

Hi. Thank you for your recent edits. An automated process has detected that when you recently edited [[Acast]], you added a link pointing to the disambiguation page [[Axios]]<!-- ([//dispenser.info.tm/~dispenser/cgi-bin/dablinks.py/Acast check to confirm]&nbsp;|&nbsp;[//dispenser.info.tm/~dispenser/cgi-bin/dab_solver.py/Acast?client=notify fix with Dab solver])-->. Such links are [[WP:INTDABLINK|usually incorrect]], since a disambiguation page is merely a list of unrelated topics with similar titles. <small>(Read the [[User:DPL bot/Dablink notification FAQ|FAQ]]{{*}} Join us at the [[Wikipedia:Disambiguation pages with links|DPL WikiProject]].)</small>

It's OK to remove this message. Also, to stop receiving these messages, follow these [[User:DPL bot|opt-out instructions]]. Thanks, [[User:DPL bot|DPL bot]] ([[User talk:DPL bot|talk]]) 05:53, 24 May 2021 (UTC)

== Your submission at [[Wikipedia:Articles for creation|Articles for creation]]: [[Masaji Ishikawa]] has been accepted ==
<div style="border:solid 1px #57DB1E; background:#E6FFE6; padding:1em; padding-top:0.5em; padding-bottom:0.5em; width:20em; color:black; margin-bottom: 1.5em; margin-left: 1.5em; width: 90%;">[[File:AFC-Logo.svg|50px|left]] '''[[Masaji Ishikawa]], which you submitted to Articles for creation, has been created.'''<br />

Congratulations, and thank you for helping expand the scope of Wikipedia! We hope you will continue making quality contributions. <br />

The article has been assessed as '''Stub-Class''', which is recorded on its [[Talk:Masaji Ishikawa|talk page]]. It is commonplace for new articles to start out as stubs and then attain higher grades as they [[Wikipedia:Article development|develop]] over time. You may like to take a look at the [[Wikipedia:WikiProject Articles for creation/Grading scheme|grading scheme]] to see how you can improve the article.

<div class="autoconfirmed-show">Since you have made at least 10 edits over more than four days, you can now [[Wikipedia:Your first article|create articles yourself]] without posting a request. However, you may continue submitting work to [[Wikipedia:Articles for creation|Articles for creation]] if you prefer.</div>

If you have any questions, you are welcome to ask at the '''<span class="plainlinks">[//en.wikipedia.org/w/index.php?title=Wikipedia:WikiProject_Articles_for_creation/Help_desk&action=edit&section=new&nosummary=1&preload=Template:AfC_talk/HD_preload&preloadparams%5B%5D=Masaji_Ishikawa help desk]</span>'''.<span class="unconfirmed-show"> Once you have made at least 10 edits and had an account for at least four days, you will have the option to [[Wikipedia:Your first article|create articles yourself]] without posting a request to [[Wikipedia:Articles for creation|Articles for creation]].</span>

If you would like to help us improve this process, please consider {{leave feedback/link|page=Wikipedia:WikiProject Articles for creation|text=leaving us some feedback}}.

Thanks again, and happy editing!
[[User:Devonian Wombat|Devonian Wombat]] ([[User talk:Devonian Wombat|talk]]) 12:53, 28 May 2021 (UTC)</div><!--Template:Afc talk-->

== Your submission at [[Wikipedia:Articles for creation|Articles for creation]]: [[Draft:DIVE Studios|DIVE Studios]] (June 7) ==
<div style="border: solid 1px #FCC; background-color: #F8EEBC; padding: 0.5em 1em; color: #000; margin: 1.5em; width: 90%;"> [[File:AFC-Logo_Decline.svg|50px|left]]Your recent article submission to [[Wikipedia:Articles for creation|Articles for Creation]] has been reviewed! Unfortunately, it has not been accepted at this time.<nowiki> </nowiki>The reason left by IceWelder was:

{{divbox|gray|3=This submission is not adequately supported by [[WP:RS|reliable sources]]. ''Reliable'' sources are required so that information can be [[WP:V|verified]]. If you need help with referencing, please see [[WP:REFB|Referencing for beginners]] and [[Wikipedia:Citing sources|Citing sources]].|}}<!--
--
-->&nbsp;The comment the reviewer left was:

{{divbox|blue|3=Sources like KpopStarz, HelloKpop, and Soompi cannot be used on Wikipedia. Aside from that, the network only appears in passing mentions ([[WP:PASSING]]) and routine news ([[WP:RUNOFTHEMILL]]) instead of in-depth coverage, as is required. See [[WP:42]] for an explainer.}} Please check the submission for any additional comments left by the reviewer.  You are encouraged to edit the submission to address the issues raised and resubmit ''when they have been resolved''.
{{clear}}
* If you would like to continue working on the submission, go to [[Draft:DIVE Studios]] and click on the "Edit" tab at the top of the window.
* If you now believe the draft cannot meet Wikipedia's standards or do not wish to progress it further, you may request deletion. Please go to [[Draft:DIVE Studios]], click on the "Edit" tab at the top of the window, add "{{tl|Db-g7}}" at the top of the draft text and click the blue "publish changes" button to save this edit.
* If you do not make any further changes to your draft, in 6 months, it will be considered abandoned and [[Wikipedia:Criteria for speedy deletion#G13. Abandoned Drafts and Articles for creation submissions|may be deleted]].
* If you need any assistance, or have experienced any [[Wikipedia:Articles for creation/Scam warning|untoward behavior]] associated with this submission, you can ask for help at the <span class="plainlinks">[//en.wikipedia.org/w/index.php?title=Wikipedia:WikiProject_Articles_for_creation/Help_desk&action=edit&section=new&nosummary=1&preload=Template:AfC_decline/HD_preload&preloadparams%5B%5D=Draft:DIVE_Studios '''Articles for creation help desk''']</span>, on the <span class="plainlinks">[//en.wikipedia.org/w/index.php?title=User_talk:IceWelder&action=edit&section=new&nosummary=1&preload=Template:AfC_decline/HD_preload&preloadparams%5B%5D=Draft:DIVE_Studios '''reviewer's talk page''']</span> or use [[Wikipedia:IRC help disclaimer|Wikipedia's real-time chat help from experienced editors]].

[[User:IceWelder|<span style="font-variant: small-caps;">IceWelder</span>]] &#91;[[User talk:IceWelder|<span style="color: #424242;">&#9993;</span>]]&#93; 12:48, 7 June 2021 (UTC)</div><!--Template:AfC decline-->

== Articles you might like to edit, from SuggestBot ==
''Note:'' All columns in this table are sortable, allowing you to rearrange the table so the articles most interesting to you are shown at the top.  All images have mouse-over popups with more information.  For more information about the columns and categories, please [[User:SuggestBot/Documentation/Suggestion columns|consult the documentation]] and please get in touch on [[User talk:SuggestBot|SuggestBot's talk page]] with any questions you might have.

{| class="wikitable sortable" border="1" style="text-align: center;"
|-
! scope="col" | Views/Day
! scope="col" | Quality
! scope="col" | Title
! scope="col" | Tagged with…

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:273}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: B, Predicted class: C|link=Death and state funeral of Kim Jong-il|Quality: Medium, Assessed class: B, Predicted class: C]]
| style="text-align: left;" | [[Death and state funeral of Kim Jong-il]] <small>([[Talk:Death and state funeral of Kim Jong-il|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:84}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Tanais|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Tanais]] <small>([[Talk:Tanais|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:23}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: B, Predicted class: GA|link=Caronia|Quality: High, Assessed class: B, Predicted class: GA]]
| style="text-align: left;" | [[Caronia]] <small>([[Talk:Caronia|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:18}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: NA, Predicted class: Start|link=Sant Martí d'Empúries|Quality: Low, Assessed class: NA, Predicted class: Start]]
| style="text-align: left;" | [[Sant Martí d'Empúries]] <small>([[Talk:Sant Martí d'Empúries|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:51}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Stari Grad, Croatia|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Stari Grad, Croatia]] <small>([[Talk:Stari Grad, Croatia|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:181}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Sciacca|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Sciacca]] <small>([[Talk:Sciacca|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:314}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: B|link=Anti-Korean sentiment|Quality: Medium, Assessed class: C, Predicted class: B]]
| style="text-align: left;" | [[Anti-Korean sentiment]] <small>([[Talk:Anti-Korean sentiment|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:44}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: B|link=The Return of Optimus Prime|Quality: Medium, Assessed class: Start, Predicted class: B]]
| style="text-align: left;" | [[The Return of Optimus Prime]] <small>([[Talk:The Return of Optimus Prime|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:491}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Helots|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Helots]] <small>([[Talk:Helots|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:149}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Sybaris|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Sybaris]] <small>([[Talk:Sybaris|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:51}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Santa Pola|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Santa Pola]] <small>([[Talk:Santa Pola|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:111}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: C, Predicted class: GA|link=Anders Tegnell|Quality: High, Assessed class: C, Predicted class: GA]]
| style="text-align: left;" | [[Anders Tegnell]] <small>([[Talk:Anders Tegnell|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:575}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: B|link=Koreans in Japan|Quality: Medium, Assessed class: C, Predicted class: B]]
| style="text-align: left;" | [[Koreans in Japan]] <small>([[Talk:Koreans in Japan|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:13}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Lisístrata|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Lisístrata]] <small>([[Talk:Lisístrata|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:283}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: C, Predicted class: GA|link=Vsauce|Quality: High, Assessed class: C, Predicted class: GA]]
| style="text-align: left;" | [[Vsauce]] <small>([[Talk:Vsauce|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:4447}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: B, Predicted class: GA|link=Spotify|Quality: High, Assessed class: B, Predicted class: GA]]
| style="text-align: left;" | [[Spotify]] <small>([[Talk:Spotify|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:9}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=WVAN-TV|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[WVAN-TV]] <small>([[Talk:WVAN-TV|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:40}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: NA, Predicted class: C|link=Negative hypergeometric distribution|Quality: Medium, Assessed class: NA, Predicted class: C]]
| style="text-align: left;" | [[Negative hypergeometric distribution]] <small>([[Talk:Negative hypergeometric distribution|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:374}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: B, Predicted class: C|link=Romanization of Greek|Quality: Medium, Assessed class: B, Predicted class: C]]
| style="text-align: left;" | [[Romanization of Greek]] <small>([[Talk:Romanization of Greek|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:1436}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Iain Stirling|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Iain Stirling]] <small>([[Talk:Iain Stirling|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:80}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Alun Cochrane|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Alun Cochrane]] <small>([[Talk:Alun Cochrane|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:3}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Start|link=Susan Shurin|Quality: Low, Assessed class: Start, Predicted class: Start]]
| style="text-align: left;" | [[Susan Shurin]] <small>([[Talk:Susan Shurin|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:9}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Managerial epidemiology|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Managerial epidemiology]] <small>([[Talk:Managerial epidemiology|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:5}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: C, Predicted class: Start|link=Respirology (journal)|Quality: Low, Assessed class: C, Predicted class: Start]]
| style="text-align: left;" | [[Respirology (journal)]] <small>([[Talk:Respirology (journal)|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:6}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Stub|link=Storyboards (album)|Quality: Low, Assessed class: Stub, Predicted class: Stub]]
| style="text-align: left;" | [[Storyboards (album)]] <small>([[Talk:Storyboards (album)|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:17}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Stub|link=Basiluzzo|Quality: Low, Assessed class: Stub, Predicted class: Stub]]
| style="text-align: left;" | [[Basiluzzo]] <small>([[Talk:Basiluzzo|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:8}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Journal of Women's Health|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Journal of Women's Health]] <small>([[Talk:Journal of Women's Health|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:40}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Fernando Simón|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Fernando Simón]] <small>([[Talk:Fernando Simón|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:29}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Stub|link=Terme|Quality: Low, Assessed class: Stub, Predicted class: Stub]]
| style="text-align: left;" | [[Terme]] <small>([[Talk:Terme|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:6}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Viveca Lärn|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Viveca Lärn]] <small>([[Talk:Viveca Lärn|talk]])</small>
| Stub
|}

SuggestBot picks articles in a number of ways based on other articles you've edited, including straight text similarity, following wikilinks, and matching your editing patterns against those of other Wikipedians.  It tries to recommend only articles that other Wikipedians have marked as needing work.  We appreciate that you have signed up to receive suggestions regularly; your contributions make Wikipedia better — thanks for helping!

If you have feedback on how to make SuggestBot better, please let us know on [[User_talk:SuggestBot|SuggestBot's talk page]]. -- [[User:SuggestBot|SuggestBot]] ([[User talk:SuggestBot|talk]]) 11:25, 16 June 2021 (UTC)

== Articles you might like to edit, from SuggestBot ==
''Note:'' All columns in this table are sortable, allowing you to rearrange the table so the articles most interesting to you are shown at the top.  All images have mouse-over popups with more information.  For more information about the columns and categories, please [[User:SuggestBot/Documentation/Suggestion columns|consult the documentation]] and please get in touch on [[User talk:SuggestBot|SuggestBot's talk page]] with any questions you might have.

{| class="wikitable sortable" border="1" style="text-align: center;"
|-
! scope="col" | Views/Day
! scope="col" | Quality
! scope="col" | Title
! scope="col" | Tagged with…

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:794}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=The Anime Man|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[The Anime Man]] <small>([[Talk:The Anime Man|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:679}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Stub, Predicted class: C|link=Gigguk|Quality: Medium, Assessed class: Stub, Predicted class: C]]
| style="text-align: left;" | [[Gigguk]] <small>([[Talk:Gigguk|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:22}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Start|link=Weighing matrix|Quality: Low, Assessed class: Start, Predicted class: Start]]
| style="text-align: left;" | [[Weighing matrix]] <small>([[Talk:Weighing matrix|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:10}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Cairns State High School|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Cairns State High School]] <small>([[Talk:Cairns State High School|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:7988}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: B|link=MrBeast|Quality: Medium, Assessed class: C, Predicted class: B]]
| style="text-align: left;" | [[MrBeast]] <small>([[Talk:MrBeast|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:109}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: C, Predicted class: GA|link=Grammis|Quality: High, Assessed class: C, Predicted class: GA]]
| style="text-align: left;" | [[Grammis]] <small>([[Talk:Grammis|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:33}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Proofs involving ordinary least squares|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Proofs involving ordinary least squares]] <small>([[Talk:Proofs involving ordinary least squares|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:27}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Moment matrix|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Moment matrix]] <small>([[Talk:Moment matrix|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:1128}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Jacobian matrix and determinant|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Jacobian matrix and determinant]] <small>([[Talk:Jacobian matrix and determinant|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:25}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Stub|link=Bidiagonal matrix|Quality: Low, Assessed class: Stub, Predicted class: Stub]]
| style="text-align: left;" | [[Bidiagonal matrix]] <small>([[Talk:Bidiagonal matrix|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:143}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Paideia|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Paideia]] <small>([[Talk:Paideia|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:65}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: B|link=European Union response to the COVID-19 pandemic|Quality: Medium, Assessed class: C, Predicted class: B]]
| style="text-align: left;" | [[European Union response to the COVID-19 pandemic]] <small>([[Talk:European Union response to the COVID-19 pandemic|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:324}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Eric Feigl-Ding|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Eric Feigl-Ding]] <small>([[Talk:Eric Feigl-Ding|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:14}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Megarian decree|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Megarian decree]] <small>([[Talk:Megarian decree|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:17}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: NA, Predicted class: C|link=Rumor spread in social network|Quality: Medium, Assessed class: NA, Predicted class: C]]
| style="text-align: left;" | [[Rumor spread in social network]] <small>([[Talk:Rumor spread in social network|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:32}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Theopompus|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Theopompus]] <small>([[Talk:Theopompus|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:6}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Start|link=Philip E. Young|Quality: Low, Assessed class: Start, Predicted class: Start]]
| style="text-align: left;" | [[Philip E. Young]] <small>([[Talk:Philip E. Young|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:20}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Polar forests of the Cretaceous|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Polar forests of the Cretaceous]] <small>([[Talk:Polar forests of the Cretaceous|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:37}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: B|link=Nigeria Centre for Disease Control|Quality: Medium, Assessed class: Start, Predicted class: B]]
| style="text-align: left;" | [[Nigeria Centre for Disease Control]] <small>([[Talk:Nigeria Centre for Disease Control|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:385}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Triangular matrix|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Triangular matrix]] <small>([[Talk:Triangular matrix|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:414}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: B|link=Polis|Quality: Medium, Assessed class: C, Predicted class: B]]
| style="text-align: left;" | [[Polis]] <small>([[Talk:Polis|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:7}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Polynomial matrix spectral factorization|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Polynomial matrix spectral factorization]] <small>([[Talk:Polynomial matrix spectral factorization|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:4}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: Start, Predicted class: GA|link=Cynthia Gómez|Quality: High, Assessed class: Start, Predicted class: GA]]
| style="text-align: left;" | [[Cynthia Gómez]] <small>([[Talk:Cynthia Gómez|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:3}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Lyda Osorio|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Lyda Osorio]] <small>([[Talk:Lyda Osorio|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:7}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Adrian Macéus|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Adrian Macéus]] <small>([[Talk:Adrian Macéus|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:24}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Stub|link=Resurreccion Acop|Quality: Low, Assessed class: Stub, Predicted class: Stub]]
| style="text-align: left;" | [[Resurreccion Acop]] <small>([[Talk:Resurreccion Acop|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:303}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Semantic Scholar|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Semantic Scholar]] <small>([[Talk:Semantic Scholar|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:238}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Stub, Predicted class: C|link=Zombie film|Quality: Medium, Assessed class: Stub, Predicted class: C]]
| style="text-align: left;" | [[Zombie film]] <small>([[Talk:Zombie film|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:25}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Hollow matrix|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Hollow matrix]] <small>([[Talk:Hollow matrix|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:26}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Exchange matrix|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Exchange matrix]] <small>([[Talk:Exchange matrix|talk]])</small>
| Stub
|}

SuggestBot picks articles in a number of ways based on other articles you've edited, including straight text similarity, following wikilinks, and matching your editing patterns against those of other Wikipedians.  It tries to recommend only articles that other Wikipedians have marked as needing work.  We appreciate that you have signed up to receive suggestions regularly; your contributions make Wikipedia better — thanks for helping!

If you have feedback on how to make SuggestBot better, please let us know on [[User_talk:SuggestBot|SuggestBot's talk page]]. -- [[User:SuggestBot|SuggestBot]] ([[User talk:SuggestBot|talk]]) 11:24, 14 July 2021 (UTC)

== Articles you might like to edit, from SuggestBot ==
''Note:'' All columns in this table are sortable, allowing you to rearrange the table so the articles most interesting to you are shown at the top.  All images have mouse-over popups with more information.  For more information about the columns and categories, please [[User:SuggestBot/Documentation/Suggestion columns|consult the documentation]] and please get in touch on [[User talk:SuggestBot|SuggestBot's talk page]] with any questions you might have.

{| class="wikitable sortable" border="1" style="text-align: center;"
|-
! scope="col" | Views/Day
! scope="col" | Quality
! scope="col" | Title
! scope="col" | Tagged with…

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:355}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Positive and negative predictive values|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Positive and negative predictive values]] <small>([[Talk:Positive and negative predictive values|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:112}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Generalized mean|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Generalized mean]] <small>([[Talk:Generalized mean|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:187}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Binary classification|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Binary classification]] <small>([[Talk:Binary classification|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:400}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: B, Predicted class: B|link=Biostatistics|Quality: Medium, Assessed class: B, Predicted class: B]]
| style="text-align: left;" | [[Biostatistics]] <small>([[Talk:Biostatistics|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:281}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: B, Predicted class: B|link=Rest area|Quality: Medium, Assessed class: B, Predicted class: B]]
| style="text-align: left;" | [[Rest area]] <small>([[Talk:Rest area|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:7}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: NA, Predicted class: Start|link=Fuzzy associative matrix|Quality: Low, Assessed class: NA, Predicted class: Start]]
| style="text-align: left;" | [[Fuzzy associative matrix]] <small>([[Talk:Fuzzy associative matrix|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:431}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Cohort study|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Cohort study]] <small>([[Talk:Cohort study|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:462}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Survival analysis|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Survival analysis]] <small>([[Talk:Survival analysis|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:106}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Biplot|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Biplot]] <small>([[Talk:Biplot|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:34}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Statistical model specification|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Statistical model specification]] <small>([[Talk:Statistical model specification|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:91}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Evaluation of binary classifiers|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Evaluation of binary classifiers]] <small>([[Talk:Evaluation of binary classifiers|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:331}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Average absolute deviation|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Average absolute deviation]] <small>([[Talk:Average absolute deviation|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:251}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Start|link=False positive rate|Quality: Low, Assessed class: Start, Predicted class: Start]]
| style="text-align: left;" | [[False positive rate]] <small>([[Talk:False positive rate|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:358}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Pharmacovigilance|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Pharmacovigilance]] <small>([[Talk:Pharmacovigilance|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:72}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Multivariate adaptive regression spline|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Multivariate adaptive regression spline]] <small>([[Talk:Multivariate adaptive regression spline|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:426}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Exponential smoothing|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Exponential smoothing]] <small>([[Talk:Exponential smoothing|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:2570}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: B, Predicted class: C|link=Podcast|Quality: Medium, Assessed class: B, Predicted class: C]]
| style="text-align: left;" | [[Podcast]] <small>([[Talk:Podcast|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:264}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Mixed model|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Mixed model]] <small>([[Talk:Mixed model|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:4}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=June E. Osborn|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[June E. Osborn]] <small>([[Talk:June E. Osborn|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:4}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Start|link=Ian Ford|Quality: Low, Assessed class: Start, Predicted class: Start]]
| style="text-align: left;" | [[Ian Ford]] <small>([[Talk:Ian Ford|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:21}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: Start, Predicted class: GA|link=Tosca Lee|Quality: High, Assessed class: Start, Predicted class: GA]]
| style="text-align: left;" | [[Tosca Lee]] <small>([[Talk:Tosca Lee|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:3}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Guosheng Yin|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Guosheng Yin]] <small>([[Talk:Guosheng Yin|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:3}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Guidances for statistics in regulatory affairs|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Guidances for statistics in regulatory affairs]] <small>([[Talk:Guidances for statistics in regulatory affairs|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:2}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Start|link=Ann M. Hardy|Quality: Low, Assessed class: Start, Predicted class: Start]]
| style="text-align: left;" | [[Ann M. Hardy]] <small>([[Talk:Ann M. Hardy|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:52}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Kotjebi|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Kotjebi]] <small>([[Talk:Kotjebi|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:10}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Stub|link=Perfect matrix|Quality: Low, Assessed class: Stub, Predicted class: Stub]]
| style="text-align: left;" | [[Perfect matrix]] <small>([[Talk:Perfect matrix|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:6}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Stub|link=Cinesias (poet)|Quality: Low, Assessed class: Stub, Predicted class: Stub]]
| style="text-align: left;" | [[Cinesias (poet)]] <small>([[Talk:Cinesias (poet)|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:13}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Emerging Infectious Diseases (journal)|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Emerging Infectious Diseases (journal)]] <small>([[Talk:Emerging Infectious Diseases (journal)|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:29}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Jo Yong-won|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Jo Yong-won]] <small>([[Talk:Jo Yong-won|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:15}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Stub|link=Signature matrix|Quality: Low, Assessed class: Stub, Predicted class: Stub]]
| style="text-align: left;" | [[Signature matrix]] <small>([[Talk:Signature matrix|talk]])</small>
| Stub
|}

SuggestBot picks articles in a number of ways based on other articles you've edited, including straight text similarity, following wikilinks, and matching your editing patterns against those of other Wikipedians.  It tries to recommend only articles that other Wikipedians have marked as needing work.  We appreciate that you have signed up to receive suggestions regularly; your contributions make Wikipedia better — thanks for helping!

If you have feedback on how to make SuggestBot better, please let us know on [[User_talk:SuggestBot|SuggestBot's talk page]]. -- [[User:SuggestBot|SuggestBot]] ([[User talk:SuggestBot|talk]]) 11:26, 11 August 2021 (UTC)

== Articles you might like to edit, from SuggestBot ==
''Note:'' All columns in this table are sortable, allowing you to rearrange the table so the articles most interesting to you are shown at the top.  All images have mouse-over popups with more information.  For more information about the columns and categories, please [[User:SuggestBot/Documentation/Suggestion columns|consult the documentation]] and please get in touch on [[User talk:SuggestBot|SuggestBot's talk page]] with any questions you might have.

{| class="wikitable sortable" border="1" style="text-align: center;"
|-
! scope="col" | Views/Day
! scope="col" | Quality
! scope="col" | Title
! scope="col" | Tagged with…

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:97}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: B|link=Prenatal testing|Quality: Medium, Assessed class: C, Predicted class: B]]
| style="text-align: left;" | [[Prenatal testing]] <small>([[Talk:Prenatal testing|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:1446}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Multi-factor authentication|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Multi-factor authentication]] <small>([[Talk:Multi-factor authentication|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:26}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: NA, Predicted class: Stub|link=Pentadiagonal matrix|Quality: Low, Assessed class: NA, Predicted class: Stub]]
| style="text-align: left;" | [[Pentadiagonal matrix]] <small>([[Talk:Pentadiagonal matrix|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:20}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Stub|link=X–Y–Z matrix|Quality: Low, Assessed class: Stub, Predicted class: Stub]]
| style="text-align: left;" | [[X–Y–Z matrix]] <small>([[Talk:X–Y–Z matrix|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:234}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Kim Il-sung University|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Kim Il-sung University]] <small>([[Talk:Kim Il-sung University|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:8}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Stub, Predicted class: C|link=Awa Marie Coll-Seck|Quality: Medium, Assessed class: Stub, Predicted class: C]]
| style="text-align: left;" | [[Awa Marie Coll-Seck]] <small>([[Talk:Awa Marie Coll-Seck|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:63}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Seasonal adjustment|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Seasonal adjustment]] <small>([[Talk:Seasonal adjustment|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:136}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: B|link=Nancy Brinker|Quality: Medium, Assessed class: Start, Predicted class: B]]
| style="text-align: left;" | [[Nancy Brinker]] <small>([[Talk:Nancy Brinker|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:167}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Isotonic regression|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Isotonic regression]] <small>([[Talk:Isotonic regression|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:40}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Adaptive clinical trial|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Adaptive clinical trial]] <small>([[Talk:Adaptive clinical trial|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:486}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: C, Predicted class: GA|link=Social distancing|Quality: High, Assessed class: C, Predicted class: GA]]
| style="text-align: left;" | [[Social distancing]] <small>([[Talk:Social distancing|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:638}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: B, Predicted class: C|link=Actuarial science|Quality: Medium, Assessed class: B, Predicted class: C]]
| style="text-align: left;" | [[Actuarial science]] <small>([[Talk:Actuarial science|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:3062}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: C, Predicted class: FA|link=Drama|Quality: High, Assessed class: C, Predicted class: FA]]
| style="text-align: left;" | [[Drama]] <small>([[Talk:Drama|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:21}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: NA, Predicted class: Start|link=Travel ban|Quality: Low, Assessed class: NA, Predicted class: Start]]
| style="text-align: left;" | [[Travel ban]] <small>([[Talk:Travel ban|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:126}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Margaret Chan|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Margaret Chan]] <small>([[Talk:Margaret Chan|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:970}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Generalized linear model|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Generalized linear model]] <small>([[Talk:Generalized linear model|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:644}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Confounding|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Confounding]] <small>([[Talk:Confounding|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:301}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Sample (statistics)|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Sample (statistics)]] <small>([[Talk:Sample (statistics)|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:798}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: B, Predicted class: C|link=Inoculation|Quality: Medium, Assessed class: B, Predicted class: C]]
| style="text-align: left;" | [[Inoculation]] <small>([[Talk:Inoculation|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:41}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Start|link=Aphaea|Quality: Low, Assessed class: Start, Predicted class: Start]]
| style="text-align: left;" | [[Aphaea]] <small>([[Talk:Aphaea|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:15}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Growth curve (statistics)|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Growth curve (statistics)]] <small>([[Talk:Growth curve (statistics)|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:2}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Start|link=Albert Vexler|Quality: Low, Assessed class: Start, Predicted class: Start]]
| style="text-align: left;" | [[Albert Vexler]] <small>([[Talk:Albert Vexler|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:9}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Rob Walling|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Rob Walling]] <small>([[Talk:Rob Walling|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:4}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Paolo Boffetta|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Paolo Boffetta]] <small>([[Talk:Paolo Boffetta|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:28}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Start|link=Strategic Advisory Group of Experts|Quality: Low, Assessed class: Start, Predicted class: Start]]
| style="text-align: left;" | [[Strategic Advisory Group of Experts]] <small>([[Talk:Strategic Advisory Group of Experts|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:20}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Start|link=Medma|Quality: Low, Assessed class: Start, Predicted class: Start]]
| style="text-align: left;" | [[Medma]] <small>([[Talk:Medma|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:7}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Seidel adjacency matrix|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Seidel adjacency matrix]] <small>([[Talk:Seidel adjacency matrix|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:6}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Stub|link=Edmonds matrix|Quality: Low, Assessed class: Stub, Predicted class: Stub]]
| style="text-align: left;" | [[Edmonds matrix]] <small>([[Talk:Edmonds matrix|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:6}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Stub|link=Stieltjes matrix|Quality: Low, Assessed class: Stub, Predicted class: Stub]]
| style="text-align: left;" | [[Stieltjes matrix]] <small>([[Talk:Stieltjes matrix|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:21}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Commutation matrix|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Commutation matrix]] <small>([[Talk:Commutation matrix|talk]])</small>
| Stub
|}

SuggestBot picks articles in a number of ways based on other articles you've edited, including straight text similarity, following wikilinks, and matching your editing patterns against those of other Wikipedians.  It tries to recommend only articles that other Wikipedians have marked as needing work.  We appreciate that you have signed up to receive suggestions regularly; your contributions make Wikipedia better — thanks for helping!

If you have feedback on how to make SuggestBot better, please let us know on [[User_talk:SuggestBot|SuggestBot's talk page]]. -- [[User:SuggestBot|SuggestBot]] ([[User talk:SuggestBot|talk]]) 11:25, 8 September 2021 (UTC)
==Concern regarding [[Draft:Siege and Storm]]==
[[File:Information.svg|25px|alt=Information icon]] Hello, Gnittopsniart. This is a bot-delivered message letting you know that [[Draft:Siege and Storm]], a page you created, has not been edited in at least 5 months.  Draft space is not an indefinite storage location for content that is not appropriate for [[WP:MAINSPACE|article space]].

If your submission is not edited soon, it could be nominated for deletion under [[WP:G13|CSD G13]].  If you would like to attempt to save it, you will need to improve it. You may request [[WP:USERFY|userfication]] of the content if it meets requirements.

If the deletion has already occurred, instructions on how you may be able to retrieve it are available [[WP:REFUND/G13|here]].

Thank you for your submission to Wikipedia. [[User:FireflyBot|FireflyBot]] ([[User talk:FireflyBot|talk]]) 03:01, 9 September 2021 (UTC)

== Articles you might like to edit, from SuggestBot ==
''Note:'' All columns in this table are sortable, allowing you to rearrange the table so the articles most interesting to you are shown at the top.  All images have mouse-over popups with more information.  For more information about the columns and categories, please [[User:SuggestBot/Documentation/Suggestion columns|consult the documentation]] and please get in touch on [[User talk:SuggestBot|SuggestBot's talk page]] with any questions you might have.

{| class="wikitable sortable" border="1" style="text-align: center;"
|-
! scope="col" | Views/Day
! scope="col" | Quality
! scope="col" | Title
! scope="col" | Tagged with…

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:37}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: B|link=Greco-Buddhist monasticism|Quality: Medium, Assessed class: Start, Predicted class: B]]
| style="text-align: left;" | [[Greco-Buddhist monasticism]] <small>([[Talk:Greco-Buddhist monasticism|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:18}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Stub, Predicted class: C|link=Nikonion|Quality: Medium, Assessed class: Stub, Predicted class: C]]
| style="text-align: left;" | [[Nikonion]] <small>([[Talk:Nikonion|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:17}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Stub, Predicted class: C|link=Center for Infectious Disease Research and Policy|Quality: Medium, Assessed class: Stub, Predicted class: C]]
| style="text-align: left;" | [[Center for Infectious Disease Research and Policy]] <small>([[Talk:Center for Infectious Disease Research and Policy|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:207}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: Start, Predicted class: GA|link=Tick-borne encephalitis|Quality: High, Assessed class: Start, Predicted class: GA]]
| style="text-align: left;" | [[Tick-borne encephalitis]] <small>([[Talk:Tick-borne encephalitis|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:390}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Block matrix|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Block matrix]] <small>([[Talk:Block matrix|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:11}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: NA, Predicted class: C|link=Field epidemiology|Quality: Medium, Assessed class: NA, Predicted class: C]]
| style="text-align: left;" | [[Field epidemiology]] <small>([[Talk:Field epidemiology|talk]])</small>
| Add sources

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:99}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Athenian festivals|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Athenian festivals]] <small>([[Talk:Athenian festivals|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:229}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Regions of ancient Greece|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Regions of ancient Greece]] <small>([[Talk:Regions of ancient Greece|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:313}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Kriging|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Kriging]] <small>([[Talk:Kriging|talk]])</small>
| Cleanup

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:82}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Stub, Predicted class: C|link=Robert Koch Institute|Quality: Medium, Assessed class: Stub, Predicted class: C]]
| style="text-align: left;" | [[Robert Koch Institute]] <small>([[Talk:Robert Koch Institute|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:21}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: NA, Predicted class: C|link=Caulonia (ancient city)|Quality: Medium, Assessed class: NA, Predicted class: C]]
| style="text-align: left;" | [[Caulonia (ancient city)]] <small>([[Talk:Caulonia (ancient city)|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:20}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Aljaraque|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Aljaraque]] <small>([[Talk:Aljaraque|talk]])</small>
| Expand

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:68}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: B|link=The Wasps|Quality: Medium, Assessed class: Start, Predicted class: B]]
| style="text-align: left;" | [[The Wasps]] <small>([[Talk:The Wasps|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:130}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: Start, Predicted class: GA|link=Giresun|Quality: High, Assessed class: Start, Predicted class: GA]]
| style="text-align: left;" | [[Giresun]] <small>([[Talk:Giresun|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:206}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: B, Predicted class: C|link=Prostitution in ancient Greece|Quality: Medium, Assessed class: B, Predicted class: C]]
| style="text-align: left;" | [[Prostitution in ancient Greece]] <small>([[Talk:Prostitution in ancient Greece|talk]])</small>
| Unencyclopaedic

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:328}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Homoscedasticity|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Homoscedasticity]] <small>([[Talk:Homoscedasticity|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:560}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: C|link=Heteroscedasticity|Quality: Medium, Assessed class: C, Predicted class: C]]
| style="text-align: left;" | [[Heteroscedasticity]] <small>([[Talk:Heteroscedasticity|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:217}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: B|link=Multilevel model|Quality: Medium, Assessed class: C, Predicted class: B]]
| style="text-align: left;" | [[Multilevel model]] <small>([[Talk:Multilevel model|talk]])</small>
| Merge

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:248}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: C, Predicted class: B|link=History of podcasting|Quality: Medium, Assessed class: C, Predicted class: B]]
| style="text-align: left;" | [[History of podcasting]] <small>([[Talk:History of podcasting|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:8}}
| {{Hs|3.0}} [[File:Stars330.svg|60 px|alt=Quality: High, Assessed class: C, Predicted class: GA|link=2013–2014 Zika virus outbreaks in Oceania|Quality: High, Assessed class: C, Predicted class: GA]]
| style="text-align: left;" | [[2013–2014 Zika virus outbreaks in Oceania]] <small>([[Talk:2013–2014 Zika virus outbreaks in Oceania|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:5}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: Start, Predicted class: C|link=Abraham Lilienfeld|Quality: Medium, Assessed class: Start, Predicted class: C]]
| style="text-align: left;" | [[Abraham Lilienfeld]] <small>([[Talk:Abraham Lilienfeld|talk]])</small>
| Wikify

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:4}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Start|link=Omar Abdi|Quality: Low, Assessed class: Start, Predicted class: Start]]
| style="text-align: left;" | [[Omar Abdi]] <small>([[Talk:Omar Abdi|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:2}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Carole Presern|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Carole Presern]] <small>([[Talk:Carole Presern|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:2}}
| {{Hs|2.0}} [[File:Stars320.svg|60 px|alt=Quality: Medium, Assessed class: NA, Predicted class: B|link=Research Centre for Emerging and Reemerging Infectious Diseases|Quality: Medium, Assessed class: NA, Predicted class: B]]
| style="text-align: left;" | [[Research Centre for Emerging and Reemerging Infectious Diseases]] <small>([[Talk:Research Centre for Emerging and Reemerging Infectious Diseases|talk]])</small>
| Orphan

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:16}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Start, Predicted class: Stub|link=Kepoi|Quality: Low, Assessed class: Start, Predicted class: Stub]]
| style="text-align: left;" | [[Kepoi]] <small>([[Talk:Kepoi|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:812}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Allan Mustafa|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Allan Mustafa]] <small>([[Talk:Allan Mustafa|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:4}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Stub|link=Annals of Clinical & Laboratory Science|Quality: Low, Assessed class: Stub, Predicted class: Stub]]
| style="text-align: left;" | [[Annals of Clinical & Laboratory Science]] <small>([[Talk:Annals of Clinical & Laboratory Science|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:9}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Ophiocordyceps camponoti-balzani|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Ophiocordyceps camponoti-balzani]] <small>([[Talk:Ophiocordyceps camponoti-balzani|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:10}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: NA, Predicted class: Stub|link=Moore matrix|Quality: Low, Assessed class: NA, Predicted class: Stub]]
| style="text-align: left;" | [[Moore matrix]] <small>([[Talk:Moore matrix|talk]])</small>
| Stub

|- style="text-align: center; height: 30px;"
| style="text-align: right;" | {{formatnum:8}}
| {{Hs|1.0}} [[File:Stars310.svg|60 px|alt=Quality: Low, Assessed class: Stub, Predicted class: Start|link=Clinical Microbiology and Infection|Quality: Low, Assessed class: Stub, Predicted class: Start]]
| style="text-align: left;" | [[Clinical Microbiology and Infection]] <small>([[Talk:Clinical Microbiology and Infection|talk]])</small>
| Stub
|}

SuggestBot picks articles in a number of ways based on other articles you've edited, including straight text similarity, following wikilinks, and matching your editing patterns against those of other Wikipedians.  It tries to recommend only articles that other Wikipedians have marked as needing work.  We appreciate that you have signed up to receive suggestions regularly; your contributions make Wikipedia better — thanks for helping!

If you have feedback on how to make SuggestBot better, please let us know on [[User_talk:SuggestBot|SuggestBot's talk page]]. -- [[User:SuggestBot|SuggestBot]] ([[User talk:SuggestBot|talk]]) 11:28, 6 October 2021 (UTC)"""

In [8]:
extract_recs(wt)

[[1, 'Greco-Buddhist monasticism', 37, 'Medium', 'Start', 'B', 'Add sources'],
 [2, 'Nikonion', 18, 'Medium', 'Stub', 'C', 'Add sources'],
 [3,
  'Center for Infectious Disease Research and Policy',
  17,
  'Medium',
  'Stub',
  'C',
  'Add sources'],
 [4, 'Tick-borne encephalitis', 207, 'High', 'Start', 'GA', 'Add sources'],
 [5, 'Block matrix', 390, 'Medium', 'Start', 'C', 'Add sources'],
 [6, 'Field epidemiology', 11, 'Medium', 'NA', 'C', 'Add sources'],
 [7, 'Athenian festivals', 99, 'Medium', 'Start', 'C', 'Cleanup'],
 [8, 'Regions of ancient Greece', 229, 'Medium', 'Start', 'C', 'Cleanup'],
 [9, 'Kriging', 313, 'Medium', 'C', 'C', 'Cleanup'],
 [10, 'Robert Koch Institute', 82, 'Medium', 'Stub', 'C', 'Expand'],
 [11, 'Caulonia (ancient city)', 21, 'Medium', 'NA', 'C', 'Expand'],
 [12, 'Aljaraque', 20, 'Low', 'Stub', 'Start', 'Expand'],
 [13, 'The Wasps', 68, 'Medium', 'Start', 'B', 'Unencyclopaedic'],
 [14, 'Giresun', 130, 'High', 'Start', 'GA', 'Unencyclopaedic'],
 [15,
  'Prosti